## Concise Logistic Regression for Image Classification

- Shows a concise implementation of logistic regression for image classification
- Uses PyTorch

In [1]:
# imports
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, models, transforms
import os
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# download the data
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip hymenoptera_data.zip

--2022-04-03 16:17:19--  https://download.pytorch.org/tutorial/hymenoptera_data.zip
Resolving download.pytorch.org (download.pytorch.org)... 13.226.230.76, 13.226.230.24, 13.226.230.114, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.226.230.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47286322 (45M) [application/zip]
Saving to: ‘hymenoptera_data.zip’

hymenoptera_data.zi 100%[===================>]  45.10M  25.3MB/s    in 1.8s    

2022-04-03 16:17:21 (25.3 MB/s) - ‘hymenoptera_data.zip’ saved [47286322/47286322]

Archive:  hymenoptera_data.zip
   creating: hymenoptera_data/
   creating: hymenoptera_data/train/
   creating: hymenoptera_data/train/ants/
  inflating: hymenoptera_data/train/ants/0013035.jpg  
  inflating: hymenoptera_data/train/ants/1030023514_aad5c608f9.jpg  
  inflating: hymenoptera_data/train/ants/1095476100_3906d8afde.jpg  
  inflating: hymenoptera_data/train/ants/1099452230_d1949d3250.jpg  
  inflating: hymenoptera_da

In [7]:
# create data loaders

data_dir = 'hymenoptera_data'

# custom transformer to flatten the image tensors
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size

    def __call__(self, img):
        result = torch.reshape(img, self.new_size)
        return result

# transformations used to standardize and normalize the datasets
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        ReshapeTransform((-1,)) # flattens the data
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        ReshapeTransform((-1,)) # flattens the data
    ]),
}

# load the correspoding folders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

# load the entire dataset; we are not using minibatches here
train_dataset = torch.utils.data.DataLoader(image_datasets['train'],
                                            batch_size=len(image_datasets['train']),
                                            shuffle=True)

test_dataset = torch.utils.data.DataLoader(image_datasets['val'],
                                           batch_size=len(image_datasets['val']),
                                           shuffle=True)

In [18]:
# build the LR model
class LR(nn.Module):
    def __init__(self, dim):
        super(LR, self).__init__()
        self.linear = nn.Linear(dim, 1)
        nn.init.zeros_(self.linear.weight)
        nn.init.zeros_(self.linear.bias)

    def forward(self, x):
        x = self.linear(x)
        x = torch.sigmoid(x)
        return x 

In [61]:
# predict function
def predict(yhat, y):
    yhat = yhat.squeeze()
    y = y.unsqueeze(0) 
    y_prediction = torch.zeros(y.size()[1])
    for i in range(yhat.shape[0]):
        if yhat[i] <= 0.5:
            y_prediction[i] = 0
        else:
            y_prediction[i] = 1
    return 100 - torch.mean(torch.abs(y_prediction - y)) * 100

In [62]:
# model config
dim = train_dataset.dataset[0][0].shape[0]

lrmodel = LR(dim).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(lrmodel.parameters(), lr=0.0001)

In [63]:
# training the model
costs = []

for ITER in range(200):
    lrmodel.train()
    x, y = next(iter(train_dataset))
    test_x, test_y = next(iter(test_dataset))

    # forward
    yhat = lrmodel.forward(x.to(device))

    cost = criterion(yhat.squeeze(), y.type(torch.FloatTensor))
    train_pred = predict(yhat, y)

    # backward
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # evaluate
    lrmodel.eval()
    with torch.no_grad():
        yhat_test = lrmodel.forward(test_x.to(device))
        test_pred = predict(yhat_test, test_y)

    if ITER % 10 == 0:
        costs.append(cost)

    if ITER % 10 == 0:
        print("Cost after iteration {}: {} | Train Acc: {} | Test Acc: {}".format(ITER, 
                                                                                    cost, 
                                                                                    train_pred,
                                                                                    test_pred))
   

Cost after iteration 0: 0.6931472420692444 | Train Acc: 50.40983581542969 | Test Acc: 45.75163269042969
Cost after iteration 10: 0.6691471338272095 | Train Acc: 64.3442611694336 | Test Acc: 54.24836730957031
Cost after iteration 20: 0.6513183116912842 | Train Acc: 68.44261932373047 | Test Acc: 54.24836730957031
Cost after iteration 30: 0.6367825269699097 | Train Acc: 68.03278350830078 | Test Acc: 54.24836730957031
Cost after iteration 40: 0.6245337128639221 | Train Acc: 69.67213439941406 | Test Acc: 54.90196228027344
Cost after iteration 50: 0.6139225959777832 | Train Acc: 70.90164184570312 | Test Acc: 56.20914840698242
Cost after iteration 60: 0.6045235991477966 | Train Acc: 72.54098510742188 | Test Acc: 56.86274337768555
Cost after iteration 70: 0.5960512161254883 | Train Acc: 74.18032836914062 | Test Acc: 57.51633834838867
Cost after iteration 80: 0.5883085131645203 | Train Acc: 73.77049255371094 | Test Acc: 57.51633834838867
Cost after iteration 90: 0.5811558365821838 | Train Acc: 

### References
- [A Logistic Regression Model from Scratch](https://colab.research.google.com/drive/1iBoJ0kngkOthy7SgVaVQA1aHEROt5mra?usp=sharing)